In [229]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time
import numpy as np

my_openweather_api_key = '8888888888888888888888888'
my_RapidAPI_key = '8888888888888888888888888888888'

In [230]:
# get timestamp in correct format for tomorrow 8 AM until 8 PM
def get_tomorrow():
    date_tomorrow = date.today() + timedelta(days=1)
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)

    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    return datetime_start,datetime_middle,datetime_end

In [241]:
def weather_forcast(list_of_cities):
   
    cities_list  = []
    weather_list = []
    error =[]
    for city in list_of_cities :
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={my_openweather_api_key}&units=metric")
        
        if city_r.status_code != 200:
            error.append(city+" got errpr code: "+ str(city_r.status_code))
            break
        
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temp"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time_3h"] = detail['dt_txt']
            weather_list.append(weather_details_dic)
    
    
    df_weather = pd.DataFrame(weather_list)
    dt_start,dt_middle,dt_end = get_tomorrow()
    df_weather['date_time_3h'] = pd.to_datetime(df_weather['date_time_3h'])
    df_weather = df_weather.loc[lambda df_ : (df_['date_time_3h'] >= dt_start) &  (df_['date_time_3h'] <= dt_end)]
    
    
    df_cities = pd.json_normalize(cities_list)
    df_cities.rename(columns={'id':'city_id'}, inplace=True)
    
    return df_weather,df_cities,error

In [242]:
def get_citys_airports(city_name, country_iso2, city_id):
    
	url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

	querystring = {"q":city_name,"limit":"10"}

	headers = {
		"X-RapidAPI-Key": my_RapidAPI_key,
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
	}
	error = []
	airports_r = requests.request("GET", url, headers=headers, params=querystring)
	empty_df = pd.DataFrame(columns=['icao','iata','airport_name','shortName','city_name','countryCode','location.lat','location.lon', 'city_id'])
	if airports_r.status_code != 200:
		error.append(city_name + " got errpr code: "+ str(airports_r.status_code))
		return empty_df,error
    
	airports_j = airports_r.json()
	if len(airports_j['items'])==0:
		error.append(city_name + " has no airport")
		return empty_df,error
	df_airports =pd.json_normalize(airports_j['items'])
	df_airports = df_airports.loc[lambda df_ : df_['countryCode'] == country_iso2]
	df_airports.rename(columns={'name':'airport_name', 'municipalityName': 'city_name'}, inplace=True)
	df_airports['city_id'] = city_id
	
	return df_airports,error

In [243]:
def get_arrival_flights(icao, city_id):

    # make api call to receive flight information
    empty_df = pd.DataFrame(columns=['flight_NO', 'flight_status', 'origin_ap_name', 'origin_ap_icao', 'origin_ap_iata', 'scheduledTimeLocal',
                                    'scheduledTimeUtc', 'terminal',
                                    'airline_name','aircraf_model', 'city_id'])
    
    dt_start,dt_middle,dt_end = get_tomorrow()
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{dt_start}/{dt_middle}"

    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": my_RapidAPI_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    arrivals_r = requests.request("GET", url, headers=headers, params=querystring)
    
    error = []

    if arrivals_r.status_code != 200:
        error.append(icao + " got errpr code: " + str(arrivals_r.status_code))
        return empty_df,error
    
    arrivals_j = arrivals_r.json()
    
    if len(arrivals_j['arrivals'])==0:
        error.append(icao + " has no flight")
        return empty_df,error
    
    arrivals_df = pd.json_normalize(arrivals_j['arrivals']).sort_values(by='movement.scheduledTimeLocal')

    
    # clean df
    # arrivals_df = arrivals_df.loc[lambda df_ : df_['isCargo']== False]
    
    try: arrivals_df = arrivals_df[['number', 'status', 
                    'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                    'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc', 'movement.terminal'
                    'airline.name', 'aircraft.model']]
    except: arrivals_df = arrivals_df[['number', 'status', 
                    'movement.airport.name','movement.airport.icao', 'movement.airport.iata', 
                    'movement.scheduledTimeLocal', 'movement.scheduledTimeUtc',
                    'airline.name', 'aircraft.model']]
   
    rename_dict= {'number':'flight_NO', 'status':'flight_status', 
                    'movement.airport.name' : 'origin_ap_name','movement.airport.icao' : 'origin_ap_icao', 'movement.airport.iata':'origin_ap_iata', 
                    'movement.scheduledTimeLocal' : 'scheduledTimeLocal', 'movement.scheduledTimeUtc' : 'scheduledTimeUtc', 'movement.terminal': 'terminal',
                    'airline.name':'airline_name', 'aircraft.model' :'aircraf_model'}
    arrivals_df.rename(columns=rename_dict , inplace=True)
    arrivals_df['city_id']=city_id

    return arrivals_df,error


In [234]:
#PART1
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']
weather_data, cities_df ,cities_error= weather_forcast(cities)

In [235]:
cities_df

,city_id,name,country,population,timezone,sunrise,sunset,coord.lat,coord.lon
0,2950159,Berlin,DE,1000000,7200,1659497333,1659552969,52.5244,13.4105
1,2988507,Paris,FR,2138551,7200,1659500781,1659554830,48.8534,2.3488
2,2759794,Amsterdam,NL,2122311,7200,1659499416,1659554976,52.3740,4.8897
3,3128760,Barcelona,ES,1621537,7200,1659502084,1659553618,41.3888,2.1590
4,4219762,Rome,US,36303,-14400,1659523981,1659573635,34.2570,-85.1647
5,2267057,Lisbon,PT,9999,3600,1659505168,1659555955,38.7167,-9.1333
6,3067696,Prague,CZ,1165581,7200,1659497631,1659552186,50.0880,14.4208
7,2761369,Vienna,AT,1000000,7200,1659497537,1659551343,48.2085,16.3721
8,3117735,Madrid,ES,1000000,7200,1659503631,1659554885,40.4165,-3.7026


In [236]:
#PART2

airports_df = pd.DataFrame(columns=['icao','iata','airport_name','shortName','city_name','countryCode','location.lat','location.lon', 'city_id'])

# airports_df = pd.concat([get_citys_airports(city['name'] ,city['country'],city['city_id']) for index,city in cities_df.iterrows()])

for index,city in cities_df.iterrows():
    city_airports, airport_error = get_citys_airports(city['name'] ,city['country'],city['city_id'])
    airports_df = pd.concat([airports_df,city_airports])
airports_df.reset_index(drop=True, inplace=True)
airports_df

,icao,iata,airport_name,shortName,city_name,countryCode,location.lat,location.lon,city_id
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889,2950159
1,EDDT,TXL,"Berlin, Berlin-Tegel",-Tegel,Berlin,DE,52.55970,13.287699,2950159
2,LFOB,BVA,"Beauvais/Tillé, Paris Beauvais Tillé",Paris Beauvais Tillé,Beauvais/Tillé,FR,49.45440,2.112780,2988507
3,LFPG,CDG,"Paris, Charles de Gaulle",Charles de Gaulle,Paris,FR,49.01280,2.549999,2988507
4,LFPO,ORY,"Paris, Paris-Orly",-Orly,Paris,FR,48.72530,2.359440,2988507
5,LFPB,LBG,"Paris, Paris-Le Bourget",-Le Bourget,Paris,FR,48.96940,2.441390,2988507
6,LFAT,LTQ,"Le Touquet-Paris-Plage, Le Touquet-Côte d'Opale",Le Touquet-Côte d'Opale,Le Touquet-Paris-Plage,FR,50.51740,1.620589,2988507
7,EHAM,AMS,"Amsterdam, Amsterdam Schiphol",Schiphol,Amsterdam,NL,52.30860,4.763889,2759794
8,LEBL,BCN,Barcelona,Barcelona,Barcelona,ES,41.29710,2.078459,3128760
9,KRME,RME,"Rome, Griffiss",Griffiss,Rome,US,43.23380,-75.407000,4219762


In [244]:
#PART3
arrival_flights_df = pd.DataFrame(columns=['flight_NO', 'flight_status', 'origin_ap_name', 'origin_ap_icao', 'origin_ap_iata','scheduledTimeLocal',
                                    'scheduledTimeUtc', 'terminal' , 'airline_name','aircraf_model', 'city_id'])
for index,airport in airports_df.iterrows():
    arrival_flights,flights_error = get_arrival_flights(airport['icao'] , airport['city_id'])
    arrival_flights_df = pd.concat([arrival_flights_df,arrival_flights])
arrival_flights_df.reset_index(drop=True, inplace=True)
arrival_flights_df

,flight_NO,flight_status,origin_ap_name,origin_ap_icao,origin_ap_iata,scheduledTimeLocal,scheduledTimeUtc,terminal,airline_name,aircraf_model,city_id
0,XQ 966,Unknown,İzmir,LTBJ,ADB,2022-08-04 05:45+02:00,2022-08-04 03:45Z,NaN,SunExpress,Boeing 737-800,2950159
1,W6 4515,Unknown,Varna,LBWN,VAR,2022-08-04 06:35+02:00,2022-08-04 04:35Z,NaN,Wizz Air,Airbus A320,2950159
2,EW 9048,Unknown,Duesseldorf,EDDL,DUS,2022-08-04 07:35+02:00,2022-08-04 05:35Z,NaN,Eurowings,Airbus A220-300,2950159
3,LH 1938,Unknown,Munich,EDDM,MUC,2022-08-04 07:35+02:00,2022-08-04 05:35Z,NaN,Lufthansa,Airbus A320,2950159
4,EW 2002,Unknown,Stuttgart,EDDS,STR,2022-08-04 07:40+02:00,2022-08-04 05:40Z,NaN,Eurowings,Airbus A319,2950159
...,...,...,...,...,...,...,...,...,...,...,...
1241,A3 700,Expected,Athens,LGAV,ATH,2022-08-04 11:45+02:00,2022-08-04 09:45Z,NaN,Aegean,Airbus A321,3117735
1242,2W 1506,Expected,Havana,MUHA,HAV,2022-08-04 11:50+02:00,2022-08-04 09:50Z,NaN,Multiple Solutions Limited,Airbus A350-900,3117735
1243,SN 3721,Expected,Brussels,EBBR,BRU,2022-08-04 11:50+02:00,2022-08-04 09:50Z,NaN,Brussels,Airbus A319,3117735
1244,IB 3485,Expected,Geneva,LSGG,GVA,2022-08-04 12:00+02:00,2022-08-04 10:00Z,NaN,Iberia,Airbus A320 NEO,3117735


In [206]:
from IPython.display import JSON
JSON(response.json())

<IPython.core.display.JSON object>